In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [6]:
def process_input(input_df):
    # deal with na values
    X = input_df.drop(['annotation_atomrec'], axis=1)
    
    # encode the categorical values
    encoded_X = X
    encoder = LabelEncoder()
    encoder.fit(X['annotation_sequence'])
    encoded_X['annotation_sequence'] = encoder.transform(X['annotation_sequence'])
    
    encoder.fit(X['entry'])
    encoded_X['entry'] = encoder.transform(X['entry'])
    
    # scale the numeric
    
    scaled_X = MinMaxScaler().fit_transform(encoded_X)
    scaled_X[: , 0] = encoded_X['annotation_sequence']
    scaled_X[: , -2] = encoded_X['entry']
    scaled_X[: , -1] = encoded_X['entry_index']
    
    return scaled_X

In [4]:
df_test = pd.read_csv("af2_dataset_testset_unlabeled.csv.gz", index_col=0)
df_test.head()

,annotation_sequence,feat_A,feat_C,feat_D,feat_E,feat_F,feat_G,feat_H,feat_I,feat_K,...,feat_DSSP_9,feat_DSSP_10,feat_DSSP_11,feat_DSSP_12,feat_DSSP_13,coord_X,coord_Y,coord_Z,entry,entry_index
0,M,False,False,False,False,False,False,False,False,False,...,0.0,0,0.0,0,0.0,33.116001,37.023998,38.417000,QCR1_HUMAN,0
1,A,True,False,False,False,False,False,False,False,False,...,-0.0,2,-0.0,0,0.0,35.849998,34.841000,40.185001,QCR1_HUMAN,1
2,A,True,False,False,False,False,False,False,False,False,...,-0.1,0,0.0,2,-0.0,37.087002,31.719999,40.547001,QCR1_HUMAN,2
3,S,False,False,False,False,False,False,False,False,False,...,-0.1,0,0.0,-2,-0.0,38.095001,28.951000,42.321999,QCR1_HUMAN,3
4,V,False,False,False,False,False,False,False,False,False,...,0.0,0,0.0,0,0.0,41.435001,27.417000,43.703999,QCR1_HUMAN,4


In [7]:
test_dataset = process_input(df_test)
print(test_dataset)

[[1.00000000e+01 0.00000000e+00 0.00000000e+00 ... 6.21994031e-01
  1.15000000e+02 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 6.29462240e-01
  1.15000000e+02 1.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 6.30991361e-01
  1.15000000e+02 2.00000000e+00]
 ...
 [1.30000000e+01 0.00000000e+00 0.00000000e+00 ... 3.24714775e-01
  9.80000000e+01 4.76000000e+02]
 [3.00000000e+00 0.00000000e+00 0.00000000e+00 ... 3.19793699e-01
  9.80000000e+01 4.77000000e+02]
 [1.10000000e+01 0.00000000e+00 0.00000000e+00 ... 3.15421759e-01
  9.80000000e+01 4.78000000e+02]]


In [8]:
filename = 'kebing_model.sav'
model = pickle.load(open(filename, 'rb'))


In [9]:
y_test_submission = model.predict(test_dataset)
print(y_test_submission)

[False False False ... False False False]


In [10]:
s = pd.Series(y_test_submission).astype(bool)
s.name = "Predicted"
s.to_csv("submission.csv")